In [34]:
import json
from pathlib import Path
from moviepy.editor import VideoFileClip
from IPython.display import Video, display

def get_cut_files(cut: Path) -> dict[str, Path]:
    return {
        'mp4': cut,
        'json': cut.parent / f"{cut.name[:-4]}.json",
        'signer': cut.parent / f"{cut.name[:-4]}_signer.json",
        'ap': cut.parent / f"{cut.name[:-4]}_ap.json"
    }

def draw_rectangle(box):
    def draw(frame):
        '''Draw a rectangle in the frame'''
        fr = frame.copy()
        bottom = int(box['y1'])
        top = int(bottom + box['height'])
        left = int(box['x1'])
        right = int(left + box['width'])
        fr[top-5:top+5, left:right] = 255 - fr[top-5:top+5, left:right]
        fr[bottom-5:bottom+5, left:right] = 255 - fr[bottom-5:bottom+5, left:right]
        fr[bottom:top, left-5:left+5] = 255 - fr[bottom:top, left-5:left+5]
        fr[bottom:top, right-5:right+5] = 255 - fr[bottom:top, right-5:right+5]
        return fr
    return draw

def draw_keypoints(keypoints, fps, size = 5, threshold = 0):
    def draw(get_frame, t):
        '''Draw keypoints in the frame'''
        fr = get_frame(t).copy()
        keypoints_t: list[float] = keypoints[min(int(t * fps), len(keypoints)-1)]['keypoints']
        it = iter(keypoints_t)
        for x, y, conf in list(zip(it, it, it))[94:]:
            if conf > threshold:
                fr[int(y)-size:int(y)+size:,int(x)-size:int(x)+size] = 255-fr[int(y)-size:int(y)+size:,int(x)-size:int(x)+size]
        return fr
    return draw

def group_kds(kds):
    '''Groups keypoint data objects that belong to same frame'''
    grouped = [[]]
    for kd in kds:
        added = False
        for g in grouped:
            if not added and (not g or g[-1]['image_id'] != kd['image_id']):
                g.append(kd)
                added = True
        if not added:
            grouped.append([kd])
    return grouped

def update_clip(clip_files: dict[str,Path], size: int, threshold: float, output: str = "temp"):
    rec_clip = VideoFileClip(str(clip_files['mp4']))
    with clip_files['signer'].open() as signerf:
        signer = json.load(signerf)
    with clip_files['ap'].open() as apf:
        ap = json.load(apf)
    signers = group_kds(ap)
    for s in signers:
        rec_clip = rec_clip.fl(draw_keypoints(s, rec_clip.fps, size, threshold))
    final_clip = rec_clip.fl_image(draw_rectangle(signer['roi']))
    final_clip.write_videofile(str(Path(f"../data/temp/{output}.mp4").resolve()), codec="libx264", fps=15, verbose=False, logger=None)
    with open(clip_files['json']) as dataf:
        data = json.load(dataf)
    return data

In [27]:
def get_score(scores: list[float]):
    m1 = max(scores)
    scores = scores.copy()
    scores.remove(m1)
    return (m1 - max(scores)) / m1


path = Path("../data/cuts/")
cuts = path.glob('**/*.mp4')

by_signers: dict[str, set[Path]] = {'all': set()}
by_scores: dict[str, set[Path]] = {'all': set()}
for cut in cuts:
    by_scores['all'].add(cut)
    by_signers['all'].add(cut)
    with get_cut_files(cut)['signer'].open() as datafile:
        line = ""
        while ']' not in line:
            line += datafile.readline()
        line = line[:-2] + '}'
    scores = json.loads(line)['scores']
    by_signers.setdefault(str(len(scores)),set()).add(cut)
    if len(scores) > 1:
        if max(scores) == 0:
            # Error en video que solo se ven imagenes en carteles
            print(cut)
            continue
        by_scores.setdefault(str(round(get_score(scores), 2)),set()).add(cut)

print(by_signers.keys())
print(len(by_scores.keys()))

../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-programa-especial-20062021/237.mp4
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-07032021/213.mp4
../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-23052021/175.mp4
../data/cuts/leyfederalsa-resumen-de-la-mesa-de-trabajo-region-patagonia/111.mp4
../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-30052021/119.mp4
../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-07062021/205.mp4
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-03012021/307.mp4
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-03012021/239.mp4
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-31012021/527.mp4
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-semana-santa-04042021/433.mp4
../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-semana-santa-04042021/448.mp4
../data/cuts/noticias-en-lengua-de-

In [35]:
SIGNERS = "all"
SCORES = "1.0"

AMOUNT = 5
OFFSET = 0

for i, c in enumerate(list(by_signers[SIGNERS] & by_scores[SCORES])[OFFSET:OFFSET + AMOUNT]):
    data = update_clip(get_cut_files(c), 7, 0.5, i)
    print(f"Video: {c}")
    print(f"Label: {data['label']}")
    display(Video(f"../data/temp/{i}.mp4", height=300))

Video: ../data/cuts/leyfederalsa-resumen-de-la-mesa-de-trabajo-region-patagonia/14.mp4
Label: gracias por tu tiempo, por esta entrevista.


Video: ../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-24102021/102.mp4
Label: de las propinas.


Video: ../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-30052021/173.mp4
Label: fam, federación argentina de musculación.


Video: ../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-13062021/133.mp4
Label: y adultez de chiron, un afroamericano que crece


Video: ../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-13062021/293.mp4
Label: la organización internacional del trabajo, oit, y unicef,


In [37]:
clip = Path(f"../data/cuts/{input('Clip:')}")

data = update_clip(get_cut_files(clip), 7, 0.5)

print(f"Video: {clip}")
print(f"Label: {data['label']}")
display(Video(str(Path("../data/temp/temp.mp4").resolve()), height=300))

Video: ../data/cuts/lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-13062021/293.mp4
Label: la organización internacional del trabajo, oit, y unicef,
